In [1]:
import json
from IPython.display import display, Image
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import Image as WidgetImage

In [2]:
def load_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

In [3]:
def save_updated_data(updated_data, start, end):
    with open(f'MB_{start}_{end}.json', 'w') as file:
        json.dump(updated_data, file)

In [4]:
class label_tool():
    def __init__(self, json_name, start, end):
        self.current_index = start
        self.data = load_data(json_name) 
        self.updated_data = []
        self.end = end
        self.start = start
    def display_content(self):
        if self.current_index < self.end:
            entry = self.data[self.current_index]

            # Load images from files and display them side by side
            with open(entry['path_input'], "rb") as file:
                image_input = file.read()
            with open(entry['path_output'], "rb") as file:
                image_output = file.read()
            

            print(f' Image ID {self.current_index}')
            
            box = widgets.HBox([
                WidgetImage(value=image_input, format='png', width=250, height=250),
                WidgetImage(value=image_output, format='png', width=250, height=250)
            ])
            display(box)
    
            # Display instruction
            print("Instruction:", entry['instruction'])

            # Display editable text area for GPT_option
            text_area = widgets.Textarea(
                value=entry['GPT_option'],
                placeholder='Type something',
                description='GPT Option:',
                disabled=False,
                layout={'width': '100%'}
            )
            
            # Display editable text area for GPT_option
            correct_area = widgets.Textarea(
                value=entry['GPT_option'].split(',')[0],
                placeholder='Type something',
                description='Answer:',
                disabled=False,
                layout={'width': '100%'}
            )
            
            display(text_area)
            display(correct_area)

            # Button to submit changes
            submit_button = widgets.Button(description="Save Changes")
            
#             # Button to double check changes
#             check_button = widgets.Button(description="Checked & Save Changes")
            
            def on_submit_clicked(b):
                # Update the entry with the new GPT_option and save it to the updated_data list
                if text_area.value != entry['GPT_option']:
                    entry['If_modified_option'] = True
                else:
                    entry['If_modified_option'] = False
                
                entry['Modified_option'] = text_area.value
                entry['Answer'] = correct_area.value

                    
                self.updated_data.append(entry)
                self.current_index += 1
                clear_output(wait=True)
                self.display_next_button()  # Display the "Show Next" button again
                

            submit_button.on_click(on_submit_clicked)
            
            
            display(submit_button)
        else:
            print("No more entries to display. Please call save_updated_data() to save results.")
            save_updated_data(self.updated_data, self.start, self.end)
    
    def save_updated_data(self):
        with open(f'MB_{self.start}_{self.end}.json', 'w') as file:
            json.dump(self.updated_data, file)
            
    def display_next_button(self):
        next_button = widgets.Button(description="Show Next")
        next_button.on_click(lambda b: self.display_content())
        display(next_button)

In [5]:
labeler = label_tool('magic_brush_test.json', 0,10)

In [6]:
labeler.display_next_button()

Button(description='Show Next', style=ButtonStyle())

 Image ID 0


Instruction: let the flowers in the vase be blue


Textarea(value='flower, cabinet, stove, oven, sink', description='GPT Option:', layout=Layout(width='100%'), p…

Textarea(value='flower', description='Answer:', layout=Layout(width='100%'), placeholder='Type something')

Button(description='Save Changes', style=ButtonStyle())

In [21]:
labeler.updated_data[2]

In [24]:
labeler.save_updated_data()

In [7]:
labeler.data

[{'path_input': 'magic_brush/test/images/125109/125109-output1.png',
  'input_global': 'A white painted kitchen with an oven and a cooker on a red tile floor',
  'path_output': 'magic_brush/test/images/125109/125109-output2.png',
  'output_local': 'blue flowers in vase on red tile floor kitchen with oven and cooker',
  'output_global': 'A white painted kitchen with an oven and a cooker on a red tile floor, featuring a blue vase of flowers.',
  'CLIP_similarity': 0.99951171875,
  'GPT_option': 'flower, cabinet, stove, oven, sink',
  'instruction': 'let the flowers in the vase be blue'},
 {'path_input': 'magic_brush/test/images/300994/300994-output1.png',
  'input_global': 'A 5th birthday cake is shaped like a fire truck.',
  'path_output': 'magic_brush/test/images/300994/300994-output2.png',
  'output_local': 'candles on fire truck cake',
  'output_global': 'A 5th birthday cake with lit candles shaped like a fire truck.',
  'CLIP_similarity': 0.99951171875,
  'GPT_option': 'cake, candle